###Delta Lake (Lakehouse Performance Optimization, Cost Saving & Best Practices)

![](/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/5_all_databricks_workouts/DELTA OPTIMIZATIONS.png)

###Performance Optimization

#### 1. OPTIMIZE COMMAND
- The OPTIMIZE command in Databricks compacts small files (due to frequent updates, merges, and streaming writes) into larger ones (~1GB) within a Delta table.
- This improves query performance by reducing the number of files that Spark needs to read and reduces metadata overhead.


In [0]:
%sql
use lakehousecat.deltadb

In [0]:
%sql
CREATE OR REPLACE TABLE tblsales
(
  sales_id INT,
  product_id INT,
  region STRING,
  sales_amount DOUBLE,
  sales_date DATE
)
USING DELTA;

In [0]:
%sql
select * from tblsales

In [0]:
%sql
INSERT INTO tblsales VALUES
  (1, 101, 'North', 1000.50, '2025-10-16'),
  (2, 102, 'South', 500.75, '2025-10-16'),
  (3, 103, 'East', 700.20, '2025-10-16'),
  (4, 104, 'West', 1200.00, '2025-10-16');

INSERT INTO tblsales VALUES
  (5, 101, 'North', 800.00, '2025-10-17'),
  (6, 102, 'South', 450.00, '2025-10-17'),
  (7, 103, 'East', 600.00, '2025-10-17'),
  (8, 104, 'West', 1100.00, '2025-10-17');


In [0]:
%sql
select * from tblsales;

In [0]:
%sql
--Check fragmentation (numFiles & sizeInBytes)
DESCRIBE DETAIL tblsales;

In [0]:
%sql
--Optimize the table
--This performs file compaction:
--Combines many small Parquet files into fewer large files (around 1 GB default).
--Improves read performance and reduces metadata overhead.
OPTIMIZE tblsales;

In [0]:
%sql
-- Verify compaction
-- After optimization, run:
DESCRIBE DETAIL tblsales;

#### 2. ZORDER
- ZORDER is an optional feature used with OPTIMIZE to colocate related data physically in the same set of files by sorting and add internal indexing for faster retrival of data.
- Reduces file scan for queries filtering on ZORDER columns using index.
- Works best for columns (low or high cardinal) used frequently in WHERE clauses.

#### EXAMPLE USE CASE:
- Periodically optimize large Delta tables with frequent writes/updates (optimize will compact)
- Use ZORDER on high-selectivity/filtering columns to improve read performance (ordering and indexing happens behind the scene).


In [0]:
%sql

-- Step 1 – Create the Delta table
use lakehousecat.deltadb;
CREATE OR REPLACE TABLE customer_txn (
    txn_id INT,
    customer_id INT,
    region STRING,
    txn_amount DOUBLE,
    txn_type STRING,
    transaction_date DATE
)
USING DELTA;

In [0]:
%sql
describe history customer_txn

In [0]:
%sql
--Step 2 – Insert multiple small batches
--Each insert writes a few small Parquet files.
-- Batch 1
INSERT INTO customer_txn VALUES
 (1, 1001, 'North', 250.00, 'Online', '2025-10-01'),
 (2, 1002, 'South', 400.00, 'Offline', '2025-10-02'),
 (3, 1003, 'West', 600.00, 'Online', '2025-10-03');

-- Batch 2
INSERT INTO customer_txn VALUES
 (4, 1001, 'North', 300.00, 'Offline', '2025-10-01'),
 (5, 1004, 'East', 750.00, 'Online', '2025-10-02'),
 (6, 1005, 'South', 180.00, 'Online', '2025-10-03');

-- Batch 3
INSERT INTO customer_txn VALUES
 (7, 1001, 'North', 270.00, 'Online', '2025-10-01'),
 (8, 1003, 'West', 500.00, 'Offline', '2025-10-02'),
 (9, 1002, 'South', 900.00, 'Online', '2025-10-03');

/*
--Before optimize or zordering
tablefolder
    - part-0
    - part-1
    - part-2
    - part-3
    - part-4
    - part-5

select * from customer_txn where region='North; --This query will scan all files and give the result of matching data

optimize customer_txn;
After optimize
tablefolder
    - part-0
    - part-1
    - part-2
    - part-3
    - part-4
    - part-5 -- All part-0 to part-4 will be kept together in part-5

select * from customer_txn where region='North; --This query will scan part-5 file alone and give the result of matching data (reduced files operation & metadata management)


optimize customer_txn zorder by transaction_date;

optimize customer_txn;
After optimize
tablefolder
    - part-0
    - part-1
    - part-2
    - part-3
    - part-4
    - part-5 -- All part-0 to part-4 will be kept together in part-5 + sort + colocate + indexing the data rows on transaction_date column

select * from customer_txn where transaction_date='2025-12-10'; --This query will scan part-5 file alone using the index and scan only the data wrt 2025-12-10 alone without querying the entire data and give the result (reduced files operation + metadata management + faster retrival of required data alone without making full table scan)
*/





In [0]:
%sql
describe history customer_txn;

In [0]:
%sql
-- Step 3 – Inspect fragmentation (numFiles & sizeInBytes)
DESCRIBE DETAIL customer_txn;

In [0]:
%sql
-- Step 4 – Run OPTIMIZE ZORDER - watch out the metrics - zOrderStats
-- Now compact and physically order data with index added.
OPTIMIZE customer_txn ZORDER BY (transaction_date);

In [0]:
%sql
--look at the operationParameters (zOrderBy)
DESCRIBE HISTORY customer_txn

In [0]:
%sql
-- Step 3 – Inspect fragmentation
DESCRIBE DETAIL customer_txn;

####3. Partitioning
Partitioning is the practice of physically splitting a table's data into separate **folders** based on a column.<br>
Good partition columns:<br>
- Low cardinality (low difference columns such as date, age, city, region, gender)
- Columns used Frequently used in filters
- Hard to manage (we can't change the partition columns very frequently)

In [0]:
%sql
use lakehousecat.deltadb;
CREATE OR REPLACE TABLE customer_txn_part1 (
    txn_id INT,
    customer_id INT,
    region STRING,
    txn_amount DOUBLE,
    txn_type STRING,
    transaction_date DATE
) 
using delta
partitioned by (transaction_date);
insert into customer_txn_part1 select * from customer_txn;
--or
--create or replace table customer_txn_part partitioned by (transaction_date) as select * from customer_txn;


In [0]:
%python
"""
create table partitioned by (transaction_date);

tablefolder
  transaction_date=2025-10-02 -- only 2025-10-02 data is present inside this folder
    - part-0
    - part-1
    - part-2
  transaction_date=2025-10-03
    - part-0
    - part-1
    - part-2
  transaction_date=2025-10-04
    - part-0
    - part-1
    - part-2
 
select * from customer_txn where transaction_date='2025-10-02';--This query will literally bring data from 2025-10-02 folder, without scanning any other folders.
"""

In [0]:
%sql
show partitions customer_txn_part1

In [0]:
%sql
explain select * from customer_txn_part1 where transaction_date='2025-10-01'; --look at the partitionfilters in the below physical plan, it used the partitioning filtering (partition pruning)
--explain select * from customer_txn_part1 where txn_type='Online';--look at the partitionfilters in the below physical plan

In [0]:
%python
#Just to show you how the data is partitioned in the filesystem (behind the scene)
spark.sql("select * from customer_txn").write.partitionBy("region").format("delta").save("/Volumes/lakehousecat/deltadb/datalake/cust_txns_partdelta",mode='overwrite')

#equivalent CTAS in Pyspark python programming
#spark.sql("select * from customer_txn").write.partitionBy("region").saveAsTable("customer_txn_part2")


In [0]:
%python
display(spark.sql('SHOW PARTITIONS customer_txn_part1'))

In [0]:
%sql
SELECT * 
FROM customer_txn_part1
WHERE transaction_date BETWEEN '2025-10-01' AND '2025-10-01';--picks the data from the 2025-10-01 folder directly and show the result quickly.

####4. Liquid Clustering
*Liquid Clustering is the* Next-generation data clustering feature that automatically manages physical data organization on disk to minimize scan cost for frequently queried columns only on Delta tables by performing automatic Optimize, Z-Ordering and Partitioning (kind of operation).<br>

While clustering in databricks delta tables does partition happens literally?
No, liquid clustering does not create literal physical partitions (subdirectories). 

Still we get the benifits of partitioning while doing clustering?
Yes, you absolutely still get the benefits of partitioning while doing clustering (where clustering will internally divide data for faster access)

While clustering in databricks delta tables does partition happens literally or it will do partition kind of operation internally?
AI Response:
In Databricks Delta Tables, liquid clustering does not create physical partitions (i.e., it does not create subdirectories like /year=2024/month=01/ on your storage). Instead, it performs a "partition-like" operation internally by reorganizing data within files and using enhanced metadata in the Delta Log to optimize data skipping

**Partition vs Liquid Clustering**
| Use case                       | Recommendation         |
| ------------------------------ | ---------------------- |
| High-cardinality columns       | Liquid clustering    |
| Frequently changing filters    | Liquid clustering    |
| Streaming / incremental loads  | Liquid clustering    |
| Static, low-cardinality (date) | Partition OR Liquid |
| Legacy Hive-style tables       | Partition           |


**Typical Use Cases**
- Large tables with frequent inserts, updates, and deletes.
- Query filtering on specific columns like customer_id, transaction_timestamp, order_date (high or low cardinal columns)

In [0]:
%sql
use lakehousecat.deltadb

###Storage, Cost & Resource Optimization

####5. Vaccum
*VACUUM* in Delta Lake removes old, unused files to free up storage, default retention hours is 168. These files come from operations like DELETE, UPDATE, or MERGE and are kept temporarily so time-travel queries can work.<br>

Before VACUUM<br>
Active + deleted parquet files exist<br>

After VACUUM<br>
Only ACTIVE parquet files remains and delete Old parquet files (from UPDATE/MERGE/DELETE) after 1 week by default or we can increase or decrease the timeline<br>
Logs remain maintained (will not delete logs, only old data deleted)<br>
Time travel beyond retention hour becomes impossible (because data is literally deleted)<br>

In [0]:
%sql
VACUUM drugstbl_merge RETAIN 168 HOURS;
--SET spark.databricks.delta.retentionDurationCheck.enabled = false;

In [0]:
%sql
-- The CLUSTER BY clause enables liquid clustering automatically.
CREATE TABLE IF NOT EXISTS sales_orders_liquid
(
  order_id INT,
  customer_id INT,
  region STRING,
  product STRING,
  quantity INT,
  price DOUBLE,
  order_date DATE
)
USING DELTA
CLUSTER BY (customer_id, region);--clustering column can be high or low cardinal, unlike partition which requires only low cardinal columns.
--column order used in cluster by is based on the primary filter, ie. whether you first filter based on customer_id or region, accordingly keep the coloumns order.

In [0]:
%sql
-- Each insert simulates separate data ingestion.

INSERT INTO sales_orders_liquid VALUES
 (1, 101, 'North', 'Laptop', 2, 65000, '2025-10-01'),
 (2, 102, 'South', 'Headphones', 5, 2500, '2025-10-01'),
 (3, 103, 'West', 'Desk Chair', 3, 4500, '2025-10-02');

INSERT INTO sales_orders_liquid VALUES
 (4, 101, 'North', 'Keyboard', 1, 1200, '2025-10-03'),
 (5, 104, 'East', 'Monitor', 2, 9500, '2025-10-03'),
 (6, 105, 'South', 'Mouse', 4, 700, '2025-10-03');


In [0]:
%sql
SELECT * FROM sales_orders_liquid where customer_id=102;


In [0]:
%sql
--check the clustering column
DESCRIBE detail sales_orders_liquid

In [0]:
%sql
UPDATE sales_orders_liquid
SET price = price * 1.05
WHERE region = 'North';

In [0]:
%sql
DESCRIBE DETAIL sales_orders_liquid

In [0]:
%sql
DESCRIBE HISTORY sales_orders_liquid;--It proves the optimize and zordering is done naturally (look at the operationmetrics column numRemovedFiles: "3")

In [0]:
%sql
DELETE FROM sales_orders_liquid
WHERE region = 'East';

In [0]:
%sql
DESCRIBE HISTORY sales_orders_liquid;

In [0]:
#I am simulating the way how delta table liquid clustering working using spark (traditional) cluster by function... This is not liquid clustering of delta tables...
spark.sql("select * from lakehousecat.deltadb.sales_orders_liquid order by region").coalesce(1).write.clusterBy("region").format("csv").save("/Volumes/lakehousecat/deltadb/datalake/cust_txns_clustercsv",mode='overwrite')

####6. Delta Table – Copy Operation

Delta Cloning allows to create a **copy of a Delta table** efficiently:
- **Full clone**: independent copy of data and metadata  
- **Shallow clone**: metadata-only copy referencing the same underlying data files  

**Clone vs CTAS**
| Aspect                  | CLONE (Delta Lake)                     | CTAS (Create Table As Select)                |
| ----------------------- | -------------------------------------- | -------------------------------------------- |
| Type                    | Delta Lake feature                     | Standard SQL feature                         |
| Data copy               | Metadata-only (Shallow) or full (Deep) | Full physical data copy                      |
| Speed                   | Very fast (especially Shallow Clone)   | Slower for large tables                      |
| Storage usage           | Minimal for Shallow Clone              | High (duplicates data)                       |
| Time travel & history   | Preserved                              | Not preserved                                |
| Schema                  | Exact copy                             | Can be modified                              |
| Dependency on source    | Shallow clone depends on source files  | Fully independent                            |
| Use case                | Dev/Test copies, backups, experiments  | Aggregations, filtered or transformed tables |
| Source table type       | Delta tables only                      | Delta or non-Delta tables                    |

##### CTAS (Create Table as Select)
No properties of parent table, but only the output of the query is created as a new table dataset..
**Full copy** creates an **independent copy**:
- Table created based on the columns & datatype returned from the select query and Data (interally files) are alone **copied**
- **No other metadata (partition/clustered) will be copied**
- No historical deltalogs copied
- CTAS can be done with the subset rows or columns data copy, rather than copying all data (Benifit) 
- Eg. If I want to copy few columns and few rows from the parent table, CTAS or Insert select will help.

In [0]:
%sql
use lakehousecat.deltadb;
CREATE TABLE sales_orders_ctas AS SELECT * FROM sales_orders_liquid where product='Keyboard';

In [0]:
%sql
describe detail sales_orders_ctas;

In [0]:
%sql
describe detail sales_orders_liquid;

In [0]:
%sql
describe history sales_orders_ctas;

In [0]:
%sql
describe history sales_orders_liquid

##### Full/Deep Clone

**Full clone** creates an **independent copy**:
- Data files are **copied**
- Medata copied
- No history is copied
- Uses more storage (because both data and metadata is copied into the new table)
- We have to clone the entire table without restricting rows or columns (we can't write query in clone)

In [0]:
%sql
CREATE or replace TABLE sales_orders_full_clone
CLONE sales_orders_liquid;
--CLONE select * from sales_orders_liquid where product='Keyboard'; --not possible

In [0]:
%sql
describe detail sales_orders_liquid

In [0]:
%sql
describe detail sales_orders_full_clone

In [0]:
%sql
describe history sales_orders_liquid

In [0]:
%sql
describe history sales_orders_full_clone

##### Shallow Clone

**Shallow clone** creates a **metadata-only copy (of the current version), no data copy**:
- Shares the same underlying data files (of the current version of the parent table)
- Very fast, uses minimal extra storage (only for metadata)
- A shallow clone shares data files, but it does NOT share the transaction log (maintained seperately)
- Even if two tables point to the same data files, they are logically independent because they have separate logs.
- If parent table is vacummed, then it affects shallow copied table also.
- If parent table modified, it will not affect child (shallow copied) table and vice versa.

In [0]:
%sql
--I am creating a deltalog to point data present in the delta file path of sales_orders_liquid
use lakehousecat.deltadb;
CREATE OR REPLACE TABLE sales_orders_shallow_clone
SHALLOW CLONE sales_orders_liquid;

In [0]:
%sql
SELECT count(1) FROM sales_orders_liquid;

In [0]:
%sql
-- Verify shallow clone
SELECT count(1) FROM sales_orders_shallow_clone;

In [0]:
%sql
DESCRIBE HISTORY sales_orders_shallow_clone;

In [0]:
%sql
--Insert into my source table will generate next version, which is not referred by the cloned table
INSERT INTO sales_orders_liquid VALUES
 (7, 101, 'North', 'Keyboard', 1, 1200, '2025-10-04');

In [0]:
%sql
UPDATE sales_orders_liquid
SET price = 200.1
WHERE region = 'South';

In [0]:
%sql
SELECT * FROM sales_orders_liquid;

In [0]:
%sql
-- Still points the old data files
SELECT * FROM sales_orders_shallow_clone;

In [0]:
%sql
--updating shallow copied table
UPDATE sales_orders_shallow_clone
SET price = 1000
WHERE region = 'North';

In [0]:
%sql
SELECT * FROM sales_orders_liquid;

In [0]:
%sql
-- Still points the old data files (from source table) & updated data from (cloned table)
SELECT * FROM sales_orders_shallow_clone;

####7. Deletion Vector (otherwise called as SOFT DELETE feature)
A Deletion Vector is a metadata structure that marks specific rows as deleted inside a Parquet file, without rewriting the file.<br>
Eg. Instead of rewriting whole files, Delta just says: “row 3, row 15, row 102 are deleted”
DV Benifits:
- Parquet file count is unchanged, only dv log files are created (Soft Delete will happen)
- New DV files exist internally

If you disable DV:
- File rewrite happens (Hard Delete will happen)
- New parquet files created
- Usecase is to ensure the deletion vector is set to True in the tables, to avoid real data deletion and file creation in the background, rather just mark the data as deleted in the DV file.


In [0]:
%sql
CREATE OR REPLACE TABLE orders_dv AS
SELECT
  id AS order_id,
  CASE WHEN id % 2 = 0 THEN 'APAC' ELSE 'EMEA' END AS region
FROM range(0, 20);
select * from orders_dv;


In [0]:
%sql
--light weight operation of not really deleting the data, rather create a dv marker
ALTER TABLE orders_dv
SET TBLPROPERTIES ('delta.enableDeletionVectors' = true);--Enabling deletion vector will enable soft delete(it will not literally delete the data in the parquet file)

In [0]:
%sql
--look at the properties delta.enableDeletionVectors: "true"
DESCRIBE DETAIL orders_dv;

In [0]:
%sql
DELETE FROM orders_dv WHERE region = 'APAC';

In [0]:
%sql
--numDeletionVectorsAdded: "8" (no real deletes happened, just deletion vector added)
DESCRIBE HISTORY orders_dv

In [0]:
%sql
ALTER TABLE orders_dv
SET TBLPROPERTIES ('delta.enableDeletionVectors' = false);

In [0]:
%sql
DELETE FROM orders_dv WHERE order_id = 3;

In [0]:
%sql
DESCRIBE DETAIL orders_dv;

In [0]:
%sql
--Check the OperationMetrics (numRemovedFiles: "1", numRemovedBytes: "922", numCopiedRows: "9", numDeletionVectorsAdded: "0", numDeletionVectorsRemoved: "0")
DESCRIBE HISTORY orders_dv;